In [ ]:
# Install dependencies
! pip install grad-cam
import pytorch_grad_cam
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget, BinaryClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
def forward(self, raw):

    # add a channel dimension to raw
    shape = tuple(raw.shape)
    raw = raw.reshape(shape[0], shape[1], shape[2], shape[3])
#    raw = raw.reshape(shape[0], 1, shape[1], shape[2])
    # compute features
    f = self.features(raw)
    f = f.view(f.size(0), -1)

    # classify
    y = self.classifier(f)

    return y

In [ ]:
# get the size of our images
for x, y in train_dataset:
  print(x.shape)
  input_size = x.shape
  break

# create the model to train
model = Vgg2D(input_size)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

# Target layer for grad cam
cam = GradCAM(model=model, target_layers=[model.features[-4]])

# create a loss
loss = torch.nn.CrossEntropyLoss()

# create an optimzer
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-4)

In [ ]:
input_tensor,label = next(iter(test_dataloader))
input_tensor,label = torch.tensor(input_tensor), torch.tensor(label)
input_tensor.to(device), label.to(device)
input_tensor = input_tensor / 255.0
input_tensor = input_tensor.unsqueeze(0).to(torch.float32)
input_tensor = input_tensor.permute(1, 0, 2,3)
input_tensor.shape

In [ ]:
def visualize_gradcam(model, input_tensor, target_class, target_layer, aug_smooth=False, eigen_smooth=False):

    # Define the GradCAM object
    cam = GradCAM(model=model, target_layers=[target_layer])

    # Define target
    targets = [ClassifierOutputTarget(target_class)]

    # Generate CAM
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets* input_tensor.size(0), aug_smooth=aug_smooth, eigen_smooth=eigen_smooth)

    # Loop through batch and show Grad-CAM
    for i in range(grayscale_cam.shape[0]):
        grayscale_cam_i = grayscale_cam[i, :]
        input_image_i = input_tensor[i].numpy().transpose(1, 2, 0)  # Convert to HWC format
        input_image_i = np.repeat(input_image_i, 3, axis=2)  # Repeat grayscale image across 3 channels

        # Ensure the image is in the correct range [0, 1]
        input_image_i = (input_image_i - input_image_i.min()) / (input_image_i.max() - input_image_i.min())

        visualization = show_cam_on_image(input_image_i, grayscale_cam_i, use_rgb=True)

        # Visualize
        plt.imshow(visualization)
        plt.axis('off')
        plt.show()

# Example usage
# Assuming you have a model and input_tensor (batch of images) ready
model = model
input_tensor = input_tensor  # Example input tensor
target_class = original_dataset.classes.index('AD')
target_layer = model.features[39]  # Experiment with different layers

visualize_gradcam(model, input_tensor, target_class, target_layer, aug_smooth=False, eigen_smooth=False)